## A simple machine learning example

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
# Create my session
spark = SparkSession.builder.appName('ML').getOrCreate()

In [10]:
# Read the dataset
training = spark.read.csv('employee.csv', header=True, inferSchema=True)

In [11]:
training.show()

+------+---+----------+------+
|  name|age|experience|salary|
+------+---+----------+------+
| Paula| 30|        10| 30000|
|  Fran| 32|         8| 25000|
|Sandra| 55|         4| 20000|
|Carlos| 24|         3| 20000|
|  Alma| 21|         1| 15000|
| Bruno| 40|         2| 10000|
+------+---+----------+------+



In [12]:
training.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



## Objective of my machine learning algorithm

Using **age** and **experience**, I will try to predict the **salary** of the employees.

First, features **age** and **experience** are grouped in one single indepedent feature using _VectorAssembler_.

In [15]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age', 'experience'], outputCol='age-exp')

In [16]:
output = featureassembler.transform(training)

In [17]:
output.show()

+------+---+----------+------+-----------+
|  name|age|experience|salary|    age-exp|
+------+---+----------+------+-----------+
| Paula| 30|        10| 30000|[30.0,10.0]|
|  Fran| 32|         8| 25000| [32.0,8.0]|
|Sandra| 55|         4| 20000| [55.0,4.0]|
|Carlos| 24|         3| 20000| [24.0,3.0]|
|  Alma| 21|         1| 15000| [21.0,1.0]|
| Bruno| 40|         2| 10000| [40.0,2.0]|
+------+---+----------+------+-----------+



In [18]:
finalized_data = output.select('age-exp','salary')

In [19]:
finalized_data.show()

+-----------+------+
|    age-exp|salary|
+-----------+------+
|[30.0,10.0]| 30000|
| [32.0,8.0]| 25000|
| [55.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [40.0,2.0]| 10000|
+-----------+------+



In [37]:
from pyspark.ml.regression import LinearRegression
# Train test split
train_data, test_data = finalized_data.randomSplit([0.7, 0.3])
regressor = LinearRegression(featuresCol='age-exp', labelCol='salary')
regressor = regressor.fit(train_data)

In [38]:
# Coefficients 
regressor.coefficients

DenseVector([7.0551, 1589.2371])

In [39]:
# Intercepts
regressor.intercept

13118.485229107868

In [40]:
# Prediction
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+----------+------+------------------+
|   age-exp|salary|        prediction|
+----------+------+------------------+
|[24.0,3.0]| 20000|18055.519305743597|
|[40.0,2.0]| 10000| 16579.16415183596|
+----------+------+------------------+



In [41]:
# Mean absolute error
pred_results.meanAbsoluteError

4261.822423046182

In [42]:
# Mean Squared error
pred_results.meanSquaredError

23533203.05356963